In [24]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import soundfile as sf
from scipy import signal

In [25]:
sgnl, fs = sf.read('sound.wav')

In [26]:
sgnl = sgnl.astype(np.float64)

In [27]:
sgnl = sgnl/np.abs(np.max(sgnl))

In [28]:
ir, fs_ir = sf.read('impulseresponse.wav')

In [29]:
ir = ir.astype(np.float64)

In [30]:
ir = ir/np.abs(np.max(ir))

In [31]:
p_max=np.argmax(np.abs(ir))

In [32]:
signal_rev = signal.fftconvolve(sgnl, ir, mode='full')    # convolving signal with impulse response

In [38]:
def shift(arr, amt):
    output = np.empty_like(arr)
    if amt >= 0:
        output[:amt] = 0.0
        output[amt:] = arr[:-amt]
    else:
        output[amt:] = 0.0
        output[:amt] = arr[-amt:]
    return output

In [39]:
signal_rev = shift(signal_rev, -p_max)    # compensating for impulse response delay

In [40]:
signal_rev = signal_rev[0:signal_clean.shape[0]]    # cut the reverberated signal to be the same length as the original

In [41]:
sf.write('reverberated_sound.wav', signal_rev, fs)